In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import bisect
from sklearn.model_selection import train_test_split

# 데이터 로드
data = pd.read_csv("C:/Users/Administrator/ship_project/model/train_std.csv")
data.shape

### 데이터 설명
# train_notnull(mean) : 결측치는 모두 항구별 평균으로 대체, 항구 1곳(RGT8)의 해당 행 데이터 삭제, 
#                       배 연령(BUILT) 이상치값을 30(upper_bound = q3 + (iqr * 1.5))으로 수정한 데이터
#                       모든 이상치값 upper lower 값으로 수정한 데이터

(391933, 28)

In [2]:
data.columns

Index(['Unnamed: 0', 'ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST', 'ID',
       'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH',
       'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN',
       'ATA_LT', 'PORT_SIZE', 'CI_HOUR', 'year', 'month', 'day', 'hour',
       'minute', 'weekday'],
      dtype='object')

In [3]:
# 데이터 생성
X = data[['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'DIST',
       'BREADTH', 'BUILT', 'DEADWEIGHT', 'DEPTH', 'DRAUGHT', 'GT', 'LENGTH',
       'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN', 'ATA_LT', 'PORT_SIZE', 
          'year', 'month', 'day', 'hour', 'minute', 'weekday']]
y = data['CI_HOUR']

In [4]:
from sklearn.model_selection import train_test_split
# 데이터를 학습용과 테스트용으로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((313546, 24), (78387, 24), (313546,), (78387,))

In [5]:
from sklearn.preprocessing import MinMaxScaler

# 변형 객체 생성
minmax_scaler = MinMaxScaler()

# 훈련데이터의 모수 분포 저장
minmax_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train_scaled = minmax_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test_scaled = minmax_scaler.transform(X_test)

In [6]:
### 사용할 모델 라이브러리 정의하기
# 선형회귀모델 : 선형방정식을 이용한 방식
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso

### 사용할 모델 라이브러리 정의하기
# 앙상블 모델 : 트리 분류 구조를 이용한 방식
#            : 여러개의 트리를 사용하는 모델
# 랜덤포레스트 : 여러개의 트리로 훈련 후 가장 선능이 좋은 트리를 사용하는 방식
from sklearn.ensemble import RandomForestRegressor

# 여러개의 트리로 훈련을 하지만, 트리와 트리는 훈련의 선능을 전달 하면서 성능을 계속 높여나가는 구조를 가짐
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

### 설치해야 사용가능한 라이브러리 : pip install xgboost
import xgboost as xgb

### 평가 라이브러리 : 회귀분석인 경우에 사용하는 평가방법(분류는 다른 평가방법 사용)
# 모델을 선택할 때 가장 비중을 많이 두는 평가는 R2(결정계수, 설명력, 예측값) 값입니다.
# 평균절대오차(MAE) : 실제값과 예측값 간의 차이를 절대값을 취하여 평균한 값
from sklearn.metrics import mean_absolute_error
# 평균제곱오차(MSE) : 실제값과 예측값 간의 차이를 제곱하여 평균한 값
from sklearn.metrics import mean_squared_error
# 결정계수(R) -> 설명력 = 예측력(훈련결과의 데이터를 잘 설명하고 있는지 여부 확인) : 설명계수
# 모델의 설명력이 좋은지 나쁜지를 나타내는 지표(예측을 잘 할 수 있는지 없는지를 결정)
# 값의 범위는 0~1 : 1에 가까울수록 설명력이 좋다고 표현(분석 보고서에 항상 들어가는 내용)
from sklearn.metrics import r2_score

In [7]:
def train_and_evalute_model(model, X_train, y_train, X_test, y_test):
    ### 모델 훈련 시키기
    model.fit(X_train, y_train)

    ### 훈련 및 검증 독립변수로 정확도 확인하기
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    ### 훈련 및 검증 데이터를 이용해서 예측 및 평가하기
    # 훈련 예측 평가
    train_pred = model.predict(X_train)
    train_mae = mean_absolute_error(y_train, train_pred)
    train_mse = mean_squared_error(y_train, train_pred)
    train_r2 = r2_score(y_train, train_pred)

    # 검증 예측 및 평가
    test_pred = model.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    # print("--------------------------[종료]----------------------------")
    return model, train_score, train_mae, train_mse, train_r2, test_score, test_mae, test_mse, test_r2

In [8]:
### 훈련에 사용할 모델 생성하기
gb_reg = GradientBoostingRegressor(random_state=42)
hgb_reg = HistGradientBoostingRegressor(random_state=42)
xgb_reg = xgb.XGBRegressor(random_state=42)

### 리스트에 담아서 반복 수행 할 수 있도록 처리
models = [gb_reg, hgb_reg, xgb_reg]
models

[GradientBoostingRegressor(random_state=42),
 HistGradientBoostingRegressor(random_state=42),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)]

In [9]:
### 함수 반복 호출하기
results = {}

for m in models :
    ### 모델은 클래스로 되었기 때문에 -> 클래스 이름을 추출 할 수 있습니다.
    model_name = m.__class__.__name__
    print(f"-----------------------------[{model_name}]-------------------------------")

    ### 함수 호출하기
    # print(train_and_evalute_model(m, X_train_scaled, y_train, X_val_scaled, y_val))
    results[model_name] = train_and_evalute_model(m, X_train_scaled, y_train, X_test_scaled, y_test)
    print()

results

-----------------------------[GradientBoostingRegressor]-------------------------------

-----------------------------[HistGradientBoostingRegressor]-------------------------------

-----------------------------[XGBRegressor]-------------------------------



{'GradientBoostingRegressor': (GradientBoostingRegressor(random_state=42),
  0.5322016780102328,
  19.525718799885627,
  889.1079187685853,
  0.5322016780102328,
  0.5322621338573419,
  19.490866767346755,
  884.0479447216769,
  0.5322621338573419),
 'HistGradientBoostingRegressor': (HistGradientBoostingRegressor(random_state=42),
  0.5720855949940646,
  18.023022041385765,
  813.3036570709352,
  0.5720855949940646,
  0.5657817536305714,
  18.132496375421606,
  820.6941880272432,
  0.5657817536305714),
 'XGBRegressor': (XGBRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_t

In [10]:
### 정확도 및 평가 결과 확인하기
for m_name, (_, train_score, train_mae, train_mse, train_r2, test_score, test_mae, test_mse, test_r2) in results.items() :
    print(f"model_name = {m_name}")
    print(f"Train Score : {train_score:.4f} / Test Score : {test_score:.4f} / Train - Test = {train_score - test_score}")
    print(f"Train MAE : {train_mae:.4f} / Test MAE : {test_mae:.4f}")
    print(f"Train MSE : {train_mse:.4f} / Test MSE : {test_mse:.4f}")
    print(f"Train R2 : {train_r2:.4f} / Test R2 : {test_r2:.4f}")
    print()

model_name = GradientBoostingRegressor
Train Score : 0.5322 / Test Score : 0.5323 / Train - Test = -6.045584710911989e-05
Train MAE : 19.5257 / Test MAE : 19.4909
Train MSE : 889.1079 / Test MSE : 884.0479
Train R2 : 0.5322 / Test R2 : 0.5323

model_name = HistGradientBoostingRegressor
Train Score : 0.5721 / Test Score : 0.5658 / Train - Test = 0.006303841363493268
Train MAE : 18.0230 / Test MAE : 18.1325
Train MSE : 813.3037 / Test MSE : 820.6942
Train R2 : 0.5721 / Test R2 : 0.5658

model_name = XGBRegressor
Train Score : 0.6132 / Test Score : 0.5822 / Train - Test = 0.030978171208265515
Train MAE : 17.2021 / Test MAE : 17.8625
Train MSE : 735.1590 / Test MSE : 789.6199
Train R2 : 0.6132 / Test R2 : 0.5822



In [11]:
### 하이퍼파라메터 튜닝 클래스 라이브러리
from sklearn.model_selection import GridSearchCV

In [12]:
### 샘플 테스트
rf_reg = HistGradientBoostingRegressor()
rf_reg

HistGradientBoostingRegressor()

In [13]:
### 하이퍼파라메터 값의 범위 지정하기
gridParams = {}
gridParams["learning_rate"] = [0.01, 0.1, 0.5]
gridParams["max_iter"] = [50, 100, 150]
gridParams["max_depth"] = [3, 5, 7]
gridParams["min_samples_leaf"] = [1, 2, 4]
gridParams["max_leaf_nodes"] = [None, 5, 10]
gridParams["l2_regularization"] = [0.0, 0.1, 0.5]

In [14]:
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error
### CV 갯수 지정하기 : 데이터를 몇개의 그룹으로 분류해서 훈련할지 지정(교차 검증 갯수)
#                     (보통 3 또는 5를 주로 사용합니다.)
cv = 5

### CPU core 갯수 지정하기
# 보통 -1을 사용합니다.(-1은 모든 core를 사용한다는 의미입니다.)
n_jobs = -1

### 평가 방법 지정 : 스스로 평가까지 진행시킵니다.(가장 좋은 성능을 만들기 위해서)
# mean_absolute_error : 평균절대오차(MAE)를 의미합니다.
# r2 :결정계수 r2_score()를 의미합니다.
# 모델 선정 기준으로 사용할 MAE를 scoring 함수에 추가합니다.
scoring = {
    'neg_mean_squared_error': make_scorer(mean_squared_error),
    'r2': make_scorer(r2_score),
    'MAE': make_scorer(mean_absolute_error)
}

### 모델 선정기준 지정 : 최종 모델 선정시 결정 기준을 평가방법 중에 지정합니다.
# 모델 선정 기준으로 사용할 MAE를 scoring 함수에 추가합니다.
# GridSearchCV 또는 cross_val_score 함수에 대해 refit으로 사용할 지표를 선택합니다.
refit = 'r2'

In [15]:
### 튜닝객체 생성하기
from sklearn.model_selection import GridSearchCV
grid_search_model = GridSearchCV(rf_reg, gridParams, scoring=scoring, refit=refit, cv=cv, n_jobs=n_jobs)
grid_search_model

GridSearchCV(cv=5, estimator=HistGradientBoostingRegressor(), n_jobs=-1,
             param_grid={'l2_regularization': [0.0, 0.1, 0.5],
                         'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 7], 'max_iter': [50, 100, 150],
                         'max_leaf_nodes': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4]},
             refit='r2',
             scoring={'MAE': make_scorer(mean_absolute_error, response_method='predict'),
                      'neg_mean_squared_error': make_scorer(mean_squared_error, response_method='predict'),
                      'r2': make_scorer(r2_score, response_method='predict')})

In [16]:
### 튜닝객체 훈련시키기(모델 훈련시키기)
grid_search_model.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=HistGradientBoostingRegressor(), n_jobs=-1,
             param_grid={'l2_regularization': [0.0, 0.1, 0.5],
                         'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 7], 'max_iter': [50, 100, 150],
                         'max_leaf_nodes': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4]},
             refit='r2',
             scoring={'MAE': make_scorer(mean_absolute_error, response_method='predict'),
                      'neg_mean_squared_error': make_scorer(mean_squared_error, response_method='predict'),
                      'r2': make_scorer(r2_score, response_method='predict')})

In [17]:
### 최적의 하이퍼파라메터와 성능지표, 최적의 모델 추출하기
print(f"1. 최적의 하이퍼파라메터(best_params_) : {grid_search_model.best_params_}")
print(f"2. 최적의 하이퍼파라메터(r2 : best_score_) : {grid_search_model.best_score_}")
print(f"3. 최적의 하이퍼파라메터(best_estimator_) : {grid_search_model.best_estimator_}")

1. 최적의 하이퍼파라메터(best_params_) : {'l2_regularization': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'max_iter': 150, 'max_leaf_nodes': None, 'min_samples_leaf': 1}
2. 최적의 하이퍼파라메터(r2 : best_score_) : 0.5799609961532607
3. 최적의 하이퍼파라메터(best_estimator_) : HistGradientBoostingRegressor(l2_regularization=0.5, max_depth=7, max_iter=150,
                              max_leaf_nodes=None, min_samples_leaf=1)


In [18]:
### joblib 라이브러리
import joblib

In [19]:
HGB_K = grid_search_model.best_estimator_

In [20]:
### 훈련 모델 저장하기
# 저장할 위치 및 파일명 정의 : 확장자는 임의로 넣어도 됩니다.
#  (보통 방식에서는 확장자 를 주로 사용합니다.)
save_path = "C:/Users/Administrator/ship_project/model/HGB_K.md"

# 모델 저장시키기
joblib.dump(HGB_K, save_path)

['C:/Users/Administrator/ship_project/model/HGB_K.md']

In [21]:
hist_model = joblib.load(save_path)
hist_model

HistGradientBoostingRegressor(l2_regularization=0.5, max_depth=7, max_iter=150,
                              max_leaf_nodes=None, min_samples_leaf=1)

In [22]:
### 예측하기
test_pred = hist_model.predict(X_test_scaled)
test_pred

array([-0.5913178 , -0.67599421, 39.50168897, ..., -0.14671303,
       -0.34752078, -0.34457469])

In [23]:
### 성능평가하기
mae = mean_absolute_error(y_test, test_pred)
mse = mean_squared_error(y_test, test_pred)
r2 = r2_score(y_test, test_pred)

mae, mse, r2

(17.72775652995137, 788.1143629715103, 0.5830193004648295)